# Simple Reasoning Chatbot Demo

This notebook demonstrates the v1 reasoning chatbot with visible thinking steps.

In [ ]:
# Setup and imports
import sys
import os
from pathlib import Path
import uuid
from datetime import datetime
from typing import List, Tuple
from IPython.display import display, Markdown, HTML

# Add parent directory to path
notebook_dir = Path.cwd()
parent_dir = notebook_dir.parent if notebook_dir.name == 'notebooks' else notebook_dir
sys.path.insert(0, str(parent_dir))

# Load environment variables
from dotenv import load_dotenv
load_dotenv()

# Import modules
from src.graph import create_graph
from src.state import State
from langchain_core.messages import HumanMessage, AIMessage

In [ ]:
# Create the graph
workflow = create_graph()

In [ ]:
# Helper class for managing sessions with colored output
class ReasoningChatbot:
    def __init__(self):
        self.workflow = create_graph()
        self.session_id = str(uuid.uuid4())
        self.history: List[Tuple[str, str]] = []
        
    def ask(self, question: str, show_reasoning: bool = True, stream_reasoning: bool = True) -> str:
        """
        Ask a question and get a reasoned response with colored output
        
        Args:
            question: The question to ask
            show_reasoning: Whether to show reasoning steps at the end
            stream_reasoning: Whether to show reasoning steps in real-time as they happen
        """
        
        # Create thread ID
        thread_id = str(uuid.uuid4())
        
        # Convert history to messages
        messages = []
        for q, a in self.history:
            messages.append(HumanMessage(content=q))
            messages.append(AIMessage(content=a))
        
        # Initialize state
        initial_state: State = {
            "session_id": self.session_id,
            "thread_id": thread_id,
            "messages": messages,
            "history": self.history,
            "current_question": question,
            "reasoning_steps": [],
            "reasoning_count": 0,
            "ready_to_answer": False,
            "context": {},
            "tools": [],
            "final_answer": None
        }
        
        # Display question
        display(Markdown(f"## Question\n{question}"))
        display(HTML("<hr style='border: 1px solid #ddd;'>"))
        
        if stream_reasoning:
            display(Markdown("### Reasoning Process"))
            
            # Stream the workflow to show reasoning in real-time
            for chunk in self.workflow.stream(initial_state):
                if 'orchestrator' in chunk:
                    # New reasoning step was added
                    updates = chunk['orchestrator']
                    if 'reasoning_steps' in updates and updates['reasoning_steps']:
                        step_num = updates.get('reasoning_count', len(updates['reasoning_steps']))
                        latest_step = updates['reasoning_steps'][-1]
                        
                        # Display reasoning in yellow/amber color
                        reasoning_html = f"""
                        <div style='background-color: #fff9e6; border-left: 4px solid #ffc107; padding: 10px; margin: 10px 0;'>
                            <strong style='color: #ff9800;'>Step {step_num}:</strong>
                            <pre style='color: #d68000; white-space: pre-wrap; font-family: monospace;'>{latest_step.content}</pre>
                        </div>
                        """
                        display(HTML(reasoning_html))
                        
                        if updates.get('ready_to_answer'):
                            display(Markdown("**Reasoning complete, generating answer...**"))
                        else:
                            display(Markdown("*Continuing to reason...*"))
                elif 'writer' in chunk:
                    # Final answer is being generated
                    updates = chunk['writer']
                    if 'final_answer' in updates:
                        final_state = updates
            
            # Get the final state
            if 'final_answer' not in locals():
                final_state = initial_state
                for chunk in self.workflow.stream(initial_state):
                    final_state.update(chunk.get('writer', chunk.get('orchestrator', {})))
        else:
            # Run the workflow
            final_state = self.workflow.invoke(initial_state)
            
            # Display reasoning steps if requested
            if show_reasoning and final_state.get("reasoning_steps"):
                display(Markdown("### Reasoning Steps"))
                for i, step in enumerate(final_state["reasoning_steps"], 1):
                    # Display reasoning in yellow/amber color
                    reasoning_html = f"""
                    <div style='background-color: #fff9e6; border-left: 4px solid #ffc107; padding: 10px; margin: 10px 0;'>
                        <strong style='color: #ff9800;'>Step {i}:</strong>
                        <pre style='color: #d68000; white-space: pre-wrap; font-family: monospace;'>{step.content if len(step.content) <= 500 else step.content[:500] + '...'}</pre>
                    </div>
                    """
                    display(HTML(reasoning_html))
        
        # Get final answer
        final_answer = final_state.get("final_answer", "No answer generated")
        
        # Update history
        self.history = final_state.get("history", self.history)
        
        # Display final answer in green
        display(HTML("<hr style='border: 1px solid #ddd; margin: 20px 0;'>"))
        answer_html = f"""
        <div style='background-color: #e8f5e9; border-left: 4px solid #4caf50; padding: 15px; margin: 10px 0;'>
            <h3 style='color: #2e7d32; margin-top: 0;'>Final Answer</h3>
            <div style='color: #1b5e20; white-space: pre-wrap; font-family: -apple-system, BlinkMacSystemFont, "Segoe UI", Roboto, sans-serif;'>{final_answer}</div>
        </div>
        """
        display(HTML(answer_html))
        
        return final_answer
    
    def reset_session(self):
        """Start a new session"""
        self.session_id = str(uuid.uuid4())
        self.history = []
        display(Markdown("**Session reset**"))

# Create chatbot instance
chatbot = ReasoningChatbot()

## Example 1: Simple Question

In [ ]:
# Simple question - should require minimal reasoning
answer = chatbot.ask("What is the capital of France?")

## Example 2: Complex Question

In [ ]:
# Complex question - should trigger more reasoning steps
answer = chatbot.ask("Explain the Black-Scholes model in simple terms, and why is it important in finance?")

## Example 3: Follow-up Question (Uses History)

In [ ]:
# Follow-up question that uses conversation context
answer = chatbot.ask("What are its main limitations?")

## Example 4: Math Problem with Streaming Reasoning

Watch the reasoning unfold in real-time for a math problem!

In [ ]:
# Math problem with real-time reasoning display
# This shows each reasoning step as it happens, not waiting until the end

answer = chatbot.ask(
    "A train travels from City A to City B at 60 mph. The return trip at 40 mph takes 2 hours longer. What's the distance between the cities?",
    stream_reasoning=True  # This enables real-time display
)

## Example 5

In [ ]:
# Same functionality but without showing reasoning
answer = chatbot.ask(
    "What's the difference between machine learning and deep learning?",
    show_reasoning=False
)

## Summary

In [ ]:
answer = chatbot.ask(
    "Sumarize our converation"
)

## Thread Lifecycle Viewer

View all messages and state changes in a thread's lifecycle:

In [ ]:
def view_thread_lifecycle(question: str):
    """
    Execute a question and display the complete thread lifecycle with colored output
    """
    display(HTML("""
    <div style='background-color: #f5f5f5; padding: 20px; border-radius: 8px;'>
        <h2 style='color: #333; margin-top: 0;'>THREAD LIFECYCLE VIEWER</h2>
    </div>
    """))
    
    display(Markdown(f"### Question\n{question}"))
    
    thread_id = str(uuid.uuid4())
    display(Markdown(f"**Thread ID:** `{thread_id}`\n\n**Session ID:** `{chatbot.session_id}`"))
    
    # Initial state
    initial_state: State = {
        "session_id": chatbot.session_id,
        "thread_id": thread_id,
        "messages": [],
        "history": chatbot.history,
        "current_question": question,
        "reasoning_steps": [],
        "reasoning_count": 0,
        "ready_to_answer": False,
        "context": {},
        "tools": [],
        "final_answer": None
    }
    
    display(Markdown("### Initial State"))
    state_info = []
    for key, value in initial_state.items():
        if key not in ['messages', 'history']:
            state_info.append(f"- **{key}**: {value}")
    display(Markdown("\n".join(state_info)))
    
    # Track all state changes
    state_changes = []
    message_log = []
    
    display(HTML("<hr style='margin: 20px 0;'>"))
    display(Markdown("### Execution Flow"))
    
    # Stream the workflow to capture all transitions
    for i, chunk in enumerate(chatbot.workflow.stream(initial_state), 1):
        node_name = list(chunk.keys())[0]
        updates = chunk[node_name]
        
        node_html = f"""
        <div style='background-color: #e3f2fd; border-left: 4px solid #2196f3; padding: 10px; margin: 10px 0;'>
            <strong style='color: #1565c0;'>Step {i}: {node_name.upper()} NODE</strong>
        """
        
        # Log state changes
        if updates:
            state_html = "<ul style='color: #424242; margin: 5px 0;'>"
            for key, value in updates.items():
                if key == 'reasoning_steps' and value:
                    state_html += f"<li><strong>{key}:</strong> Added step {len(value)}</li>"
                    # Store reasoning content
                    for step in value:
                        if hasattr(step, 'content'):
                            message_log.append(('reasoning', step.content))
                elif key == 'messages' and value:
                    state_html += f"<li><strong>{key}:</strong> {len(value)} messages</li>"
                elif key == 'final_answer':
                    state_html += f"<li><strong>{key}:</strong> Generated ({len(str(value))} chars)</li>"
                    message_log.append(('answer', value))
                else:
                    state_html += f"<li><strong>{key}:</strong> {value}</li>"
            state_html += "</ul>"
            node_html += state_html
            
            state_changes.append({
                'node': node_name,
                'updates': updates
            })
        
        node_html += "</div>"
        display(HTML(node_html))
    
    display(HTML("<hr style='margin: 20px 0;'>"))
    display(Markdown("### All Messages Generated"))
    
    for msg_type, content in message_log:
        if msg_type == 'reasoning':
            # Yellow/amber for reasoning
            reasoning_html = f"""
            <div style='background-color: #fff9e6; border-left: 4px solid #ffc107; padding: 10px; margin: 10px 0;'>
                <strong style='color: #ff9800;'>Reasoning Step:</strong>
                <pre style='color: #d68000; white-space: pre-wrap; font-family: monospace;'>{content[:500] + '...' if len(content) > 500 else content}</pre>
            </div>
            """
            display(HTML(reasoning_html))
        elif msg_type == 'answer':
            # Green for final answer
            answer_html = f"""
            <div style='background-color: #e8f5e9; border-left: 4px solid #4caf50; padding: 10px; margin: 10px 0;'>
                <strong style='color: #2e7d32;'>Final Answer:</strong>
                <pre style='color: #1b5e20; white-space: pre-wrap; font-family: monospace;'>{content[:500] + '...' if len(content) > 500 else content}</pre>
            </div>
            """
            display(HTML(answer_html))
    
    # Get final state
    final_state = initial_state
    for change in state_changes:
        final_state.update(change['updates'])
    
    display(HTML("<hr style='margin: 20px 0;'>"))
    display(Markdown("### Final State"))
    
    final_state_info = []
    for key, value in final_state.items():
        if key == 'reasoning_steps':
            final_state_info.append(f"- **{key}**: {len(value)} steps")
        elif key == 'messages':
            final_state_info.append(f"- **{key}**: {len(value)} messages")
        elif key == 'history':
            final_state_info.append(f"- **{key}**: {len(value)} Q&A pairs")
        elif key == 'final_answer':
            final_state_info.append(f"- **{key}**: {len(value)} characters" if value else f"- **{key}**: None")
        else:
            final_state_info.append(f"- **{key}**: {value}")
    display(Markdown("\n".join(final_state_info)))
    
    display(HTML("<hr style='margin: 20px 0;'>"))
    
    # Statistics with nice formatting
    stats_html = f"""
    <div style='background-color: #f3e5f5; border-left: 4px solid #9c27b0; padding: 15px; margin: 10px 0;'>
        <h3 style='color: #6a1b9a; margin-top: 0;'>Statistics</h3>
        <ul style='color: #4a148c;'>
            <li><strong>Total State Transitions:</strong> {len(state_changes)}</li>
            <li><strong>Reasoning Steps:</strong> {final_state.get('reasoning_count', 0)}</li>
            <li><strong>Nodes Executed:</strong> {', '.join(set(c['node'] for c in state_changes))}</li>
            <li><strong>Thread Completed:</strong> {'Yes' if final_state.get('final_answer') else 'No'}</li>
        </ul>
    </div>
    """
    display(HTML(stats_html))
    
    # Update history
    chatbot.history = final_state.get("history", chatbot.history)
    
    return final_state

# Example: View the complete lifecycle of a reasoning thread
result = view_thread_lifecycle("What are the key differences between TCP and UDP protocols?")